In [1]:
from datetime import datetime
t1=datetime.strptime('2020/1/20','%Y/%m/%d')
t1
t2=datetime.strptime('2025/1/20','%Y/%m/%d')
t2
t2-t1

datetime.timedelta(days=1827)

In [2]:
import numpy as np
np.timedelta64(1,'D')

numpy.timedelta64(1,'D')

In [3]:
np.timedelta64(1,'W')/np.timedelta64(1,'D')

7.0

In [11]:
2+3

5

In [79]:
#dataframe计算#
import pandas as pd
data=pd.DataFrame({'a':[1,3,6,9,0,3],'b':[3,5,1,4,11,3],'c':[2,6,3,8,1,5]})
corr=data.corr(method='pearson')
corr


,a,b,c
a,1.000000,-0.522841,0.759255
b,-0.522841,1.000000,-0.340468
c,0.759255,-0.340468,1.000000


In [88]:

def get_corr(data,method='pearson',threshold=0):
    '''
    程序功能：
    相关系数排序
    根据阈值选择相关系数
     
    参数输入及选择：
    data:DataFrame格式
    method='pearson','spearman','kendall'

    用途：根据相关系数来选择特征
    '''

    import pandas as pd
    corr=data.corr(method=method)
    col_list=list(corr.columns)
    records=[]
    for x in col_list:
        x=col_list.pop(0)
        if len(col_list)>0:
            for y in col_list:
                records.append([x,y,corr.loc[x,y]])
    df = pd.DataFrame(records,columns=['X','Y','Corr']).sort_values('Corr',ascending=False).reset_index()
    df=df[abs(df['Corr'])>threshold]
    return df

In [89]:
corr_list=get_corr(data,threshold=0)
corr_list

,index,X,Y,Corr
0,1,a,c,0.759255
1,2,b,c,-0.340468
2,0,a,b,-0.522841


In [35]:
records

[['b', 'c', -0.3404675279523206]]

In [18]:
(corr<1) & (corr>0.5)

,a,b,c
a,False,False,True
b,False,False,False
c,True,False,False


In [5]:
corr.dtypes

a    float64
b    float64
dtype: object

In [9]:
df = pd.DataFrame(np.random.randn(10, 2))
df


,0,1
0,-1.322850,0.668215
1,-1.208507,-0.248276
2,-1.669636,0.451794
3,-0.651485,0.768271
4,-0.493868,0.267296
5,1.491827,0.449765
6,-1.859930,0.294492
7,-0.112099,-0.069439
8,0.107107,0.059002
9,1.650947,0.090039


In [10]:
df[np.random.rand(df.shape[0]) > 0.5] = 1.5
df

,0,1
0,-1.322850,0.668215
1,-1.208507,-0.248276
2,1.500000,1.500000
3,-0.651485,0.768271
4,1.500000,1.500000
5,1.500000,1.500000
6,1.500000,1.500000
7,1.500000,1.500000
8,1.500000,1.500000
9,1.650947,0.090039


In [90]:
def add5(v):
    return v+5
map(add5,range(5))

In [92]:
list(map(add5,range(5)))

[5, 6, 7, 8, 9]

In [95]:
from sklearn.datasets import make_regression
from scipy.stats import pearsonr
X,Y = make_regression(n_samples=1000, n_features=3, n_informative=1, noise=100, random_state=9527)


In [97]:
print(pearsonr(X[:,0],Y),pearsonr(X[:,1],Y),pearsonr(X[:,2],Y))

(0.01293680050695129, 0.6828310401786694) (0.6680920624164118, 2.8345376164035335e-130) (0.03938982451397195, 0.21330062660673496)


In [101]:
X.shape

(1000, 3)

In [122]:
np.array(list(map(lambda x: pearsonr(x, Y)[0], X.T)))


array([0.0129368 , 0.66809206, 0.03938982])

In [129]:
from sklearn.feature_selection import SelectKBest
#SelectKBest(lambda X, Y: np.array(list(map(lambda x: pearsonr(x, Y)[0], X.T))).T, k=2).fit_transform(X,Y)
SelectKBest(lambda X,Y: np.array(list(map(lambda x: pearsonr(x, Y)[0], X.T))), k=2)

SelectKBest(k=2, score_func=<function <lambda> at 0x000001F8E6B3DF78>)

In [121]:
SelectKBest(lambda X, Y: np.array(list(map(lambda x: pearsonr(x, Y)[0], X.T))), k=2).fit_transform(X,Y)

array([[-0.6872076 , -0.60472765],
       [ 2.31728703, -1.44588579],
       [-1.51368674,  0.01249735],
       ...,
       [-0.77685783, -0.86954534],
       [ 0.38121901, -0.38671773],
       [ 0.70171457,  0.7193668 ]])

In [133]:
'''
运用相关系数法对回归特征的选择

运用SelectKBest类创建对象修改score_func参数，选择需要的函数或者放入自己写的函数，
k值选择预期的参数或者进行参数搜索，即可完成一定需求的特征选择。
'''

from sklearn import datasets

iris = datasets.load_iris()
#X=iris.data
#Y = iris.target
selector=SelectKBest(lambda X, Y: np.array(list(map(lambda x: pearsonr(x, Y)[0], X.T))), k=2)
selected_data = pd.DataFrame(selector.fit_transform(iris.data, iris.target),\
                             columns=np.array(iris.feature_names)\
                             [selector.get_support(indices=True)])  
selected_data

,petal length (cm),petal width (cm)
0,1.4,0.2
1,1.4,0.2
2,1.3,0.2
3,1.5,0.2
4,1.4,0.2
...,...,...
145,5.2,2.3
146,5.0,1.9
147,5.2,2.0
148,5.4,2.3


In [161]:
from sklearn.feature_selection import VarianceThreshold
from sklearn import datasets

iris = datasets.load_iris()
selectorV = VarianceThreshold(threshold=0.5)
V_data = selectorV.fit_transform(iris.data)
select_name_index0 = selectorV.get_support(indices=True)# 所有留下特征的索引值

features = pd.DataFrame(V_data,columns=np.array(iris.feature_names)\
                        [selectorV.get_support(indices=True)])
features
#iris.feature_names
#np.array(iris.feature_names)

,sepal length (cm),petal length (cm),petal width (cm)
0,5.1,1.4,0.2
1,4.9,1.4,0.2
2,4.7,1.3,0.2
3,4.6,1.5,0.2
4,5.0,1.4,0.2
...,...,...,...
145,6.7,5.2,2.3
146,6.3,5.0,1.9
147,6.5,5.2,2.0
148,6.2,5.4,2.3
